## 1. The load of packages, modules, and data

In [1]:
import sys
from aqua import Reader
sys.path.insert(0, '../../')
from tropical_rainfall import Tropical_Rainfall

In [2]:
from aqua.slurm import slurm

In [3]:
slurm.job(cores=16, memory="250 GB")

2023-08-16 15:11:03 :: create_folder :: WARNING  -> Folder ./slurm already exists
2023-08-16 15:11:03 :: create_folder :: WARNING  -> Folder ./slurm/logs already exists
2023-08-16 15:11:03 :: create_folder :: WARNING  -> Folder ./slurm/output already exists
2023-08-16 15:11:15 :: slurm :: WARNING  -> #!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=16
#SBATCH --mem=233G
#SBATCH -t 02:30:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out

/work/bb1153/b382267/mamba_dir/mambaforge/envs/tropical-rainfall_m/bin/python -m distributed.cli.dask_worker tcp://136.172.124.6:39149 --nthreads 4 --nworkers 4 --memory-limit 58.21GiB --name dummy-name --nanny --death-timeout 60



In [4]:
slurm.squeue()

JOBID      CPUS  NODES ST         NAME                 TIME       START_TIME           DEPENDENCY           PARTITION            MIN_MEMORY          
6519869    16    1     PD         dask-worker          0:00       N/A                  (null)               compute              233G                


0

In [5]:
diag = Tropical_Rainfall(trop_lat=15,  num_of_bins = 1000, first_edge = 0, width_of_bin = 1 - 10**(-6), loglevel='debug')

In [6]:
reader              = Reader(model="IFS", exp="tco1279-orca025-cycle3", source="2D_1h_native", regrid="r010")
ifs_tco1279_orca025_retrived = reader.retrieve()

In [ ]:
ifs_tco1279_orca025_retrived = ifs_tco1279_orca025_retrived.isel(time=slice(0, 10))
ifs_tco1279_orca025 = reader.regrid(ifs_tco1279_orca025_retrived)

In [ ]:
ifs_tco1279_orca025.lat.size*0.005, ifs_tco1279_orca025.lon.size*0.005

In [ ]:
path_to_netcdf = "/work/bb1153/b382267/tropical_rainfall_cicle3/NetCDF/UTC_timing/"

In [ ]:
diag.add_UTC_DataAaray(data = ifs_tco1279_orca025_retrived, space_grid_factor = 0.005,
                        path_to_netcdf=path_to_netcdf, name_of_file = 'TEST_high_res_icon_ngc3028')

## OR low-res models

reader          = Reader(model="ICON", exp="ngc3028", source="lra-r100-monthly")
icon_ngc3028    = reader.retrieve()
icon_ngc3028    = icon_ngc3028['tprate'][:]*86400
icon_ngc3028.attrs['units'] = "mm/day"

In [ ]:
diag.add_UTC_DataAaray(data = icon_ngc3028,  space_grid_factor = 0.05, time_length=2,
                              path_to_netcdf=path_to_netcdf, name_of_file = 'TEST_icon_ngc3028')

In [ ]:
diag.open_dataset("/work/bb1153/b382267/tropical_rainfall_cicle3/NetCDF/UTC_timing/trop_rainfall_TEST_icon_ngc3028_2020-01-21T00_2020-02-01T00_histogram.nc")

In [ ]:
utc_time = temp['utc_time'].stack(total=['time', 'lat', 'lon'])
tprate = temp['tprate'].stack(total=['time', 'lat', 'lon'])
new_data = [utc_time, tprate]